In [47]:
import os
import glob
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook as tqdm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import utils
from constants import DATA_DIR

In [66]:
# Linear Regression with basic 9 variables counted for fantasy points
def pred (df, weight):
    df.fillna(0, inplace=True)
    #pred_columns = ['FG%','FT%','3P','PTS','TRB','AST','STL','BLK','TOV']
    X = df.loc[:, pred_columns]
    X = df.loc[:, df.columns != 'VALUE']._get_numeric_data()
    X = MinMaxScaler().fit_transform(X)
    y = df['VALUE']
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=None)
    reg = LinearRegression()
    scores = cross_val_score(reg, X_train, y_train, scoring='r2', cv=2)
    print(scores)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    print(mae(y_test, y_pred))
    print(mse(y_test, y_pred, squared=False))
    print(r2(y_test, y_pred))
    print('\n')

#errors = utils.cross_val(reg, X, y, n_folds=5, verbose=0)
#utils.summarize_errors(errors)

In [85]:
weights = ['base', 'linear', 'quad', 'sqrt']
seasons = ['2016-17','2017-18','2018-19','2019-20']

for weight in weights:
    df = pd.DataFrame()
    for season in seasons:
        df1 = pd.read_csv(os.path.join(DATA_DIR,'Weighted',weight,season+'.csv'))
        df = df.append(df1)
    print(weight)
    print(df.info())
    pred(df, line)

base
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1083 entries, 0 to 264
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   PLAYER  1083 non-null   object 
 1   POS     1083 non-null   object 
 2   AGE     1083 non-null   int64  
 3   TEAM    1083 non-null   object 
 4   G       1083 non-null   float64
 5   GS      1083 non-null   int64  
 6   MP      1083 non-null   float64
 7   FG      1083 non-null   float64
 8   FGA     1083 non-null   float64
 9   FG%     1081 non-null   float64
 10  3P      1083 non-null   float64
 11  3PA     1083 non-null   float64
 12  3P%     1035 non-null   float64
 13  2P      1083 non-null   float64
 14  2PA     1083 non-null   float64
 15  2P%     1081 non-null   float64
 16  eFG%    1083 non-null   float64
 17  FT      1083 non-null   float64
 18  FTA     1083 non-null   float64
 19  FT%     1063 non-null   float64
 20  ORB     1083 non-null   float64
 21  DRB     1083 non-null   float64
 

In [88]:
wow = pd.read_csv(os.path.join(DATA_DIR,'Weighted','sqrt','2020-21.csv'))
wow = wow.set_index('PLAYER').drop(['VALUE'], axis=1)._get_numeric_data().fillna(0)
noice = MinMaxScaler().fit_transform(wow)
asfd = reg.predict(noice)
wow['VALUE'] = asfd.tolist()
wow.to_csv('noice.csv')
wow.head()

,AGE,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,VALUE
PLAYER,,,,,,,,,,,,,,,,,,,,,
Bogdan Bogdanović,28,68.169809,27,29.7,6.1,12.8,0.433943,2.185954,7.6,0.438,...,0.5,3.2,3.313518,3.512314,0.975882,0.3,1.675882,2.3,13.963021,6.509702
Clint Capela,26,56.991612,63,30.1,6.6,11.0,0.641028,0.000000,0.0,0.000,...,4.7,9.6,12.701268,1.195862,0.765892,2.0,1.483548,2.3,14.820920,5.131435
John Collins,23,55.780599,63,29.3,6.8,12.2,0.573467,0.940042,3.3,0.399,...,1.9,5.5,9.322369,1.622304,0.615331,1.0,1.771744,3.1,18.206621,4.186731
Kris Dunn,26,49.535774,0,11.3,0.3,3.0,0.433902,0.682344,0.5,0.000,...,0.0,1.5,3.939367,4.913882,1.829459,0.5,2.026971,2.5,10.135529,-3.357210
Danilo Gallinari,32,54.840232,4,24.0,4.0,9.3,0.435880,2.416654,5.0,0.406,...,0.3,3.8,5.370546,2.137538,0.673933,0.2,1.287707,2.1,18.076837,3.786186
